# TeleQnA

## Create vector Store

In [1]:
!python ../scripts/create_vector_store.py \
  --dataset "teleqna" \
  --emb_model "BAAI/bge-small-en-v1.5" \
  --cs 150 \
  --co 20 \
  --bs_emb 2048 \
  --output_dir "../vector_stores/teleqna/base_"

Load model embedding
Using device: cuda
Total number of documents: 554
Creating passages
100%|█████████████████████████████████████████| 554/554 [10:41<00:00,  1.16s/it]
Number of documents: 389606
Removing duplicate passages
Total number of passages created: 378571
Creating vector store
Load model embedding : BAAI/bge-small-en-v1.5
Using device: cuda
Generando embeddings: 100%|███████████████████| 185/185 [05:32<00:00,  1.80s/it]
✅ Índice FAISS creado exitosamente.
💾 Vector store saved in ../vector_stores/teleqna/base_vs_teleqna_150_20


## Training embedding

In [2]:
!python  ../scripts/train_embedding.py \
  --name_dataset "teleqna" \
  --model_name "BAAI/bge-small-en-v1.5" \
  --new_model_name "bge-small-3gpp" \
  --epochs 10 \
  --batch_size 128 \
  --output_dir "../models/teleqna/embedding/"

wandb: Currently logged in as: dinho15971 (dinho15971-unicamp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Starting main process...
Loading dataset splits for teleqna
Train: 724
Val: 181
Test: 905
Datasets loaded and prepared.
Datasets loaded and prepared.
Creating evaluator...
Evaluator created.
wandb: Tracking run with wandb version 0.19.10
wandb: Run data is saved locally in /local1/ronaldinho/projects/test_sbbd/notebooks2/wandb/run-20250503_163419-5856s1rq
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bge-small-3gpp_10e_128bs
wandb: ⭐️ View project at https://wandb.ai/dinho15971-unicamp/SBBD_embeddings
wandb: 🚀 View run at https://wandb.ai/dinho15971-unicamp/SBBD_embeddings/runs/5856s1rq
Loading model: BAAI/bge-small-en-v1.5
Model and loss function initialized.
Configuring training arguments...
Training arguments configured.
Starting training process...
{'loss': 0.3891, 'grad_norm': 1.2274510860443115, 'learning_rate': 4.73408160080853

## Evaluate Embeddings

In [3]:
!python  ../scripts/evaluate_embedding.py \
  --name_dataset "teleqna" \
  --output_dir "../results/teleqna/" \
  --models_dir "../models/teleqna/embedding/"

Cargando modelos...
Models loaded
Loading datasets...
Loading dataset splits for teleqna
Train: 724
Val: 181
Test: 905
Datasets loaded and prepared.
Datasets loaded and prepared.
Loaded dataset
Creating evaluator...
Evaluator created.
Evaluating models
Save results...


## Create new vector Store

In [4]:
!python ../scripts/create_vector_store.py \
  --dataset "teleqna" \
  --emb_model "../models/teleqna/embedding/bge-small-3gpp_10e_128bs/checkpoint-30" \
  --cs 150 \
  --co 20 \
  --bs_emb 2048 \
  --output_dir "../vector_stores/teleqna/ft_"

Load model embedding
Using device: cuda
Total number of documents: 554
Creating passages
100%|█████████████████████████████████████████| 554/554 [10:44<00:00,  1.16s/it]
Number of documents: 389606
Removing duplicate passages
Total number of passages created: 378571
Creating vector store
Load model embedding : ../models/teleqna/embedding/bge-small-3gpp_10e_128bs/checkpoint-30
Using device: cuda
Generando embeddings: 100%|███████████████████| 185/185 [05:36<00:00,  1.82s/it]
✅ Índice FAISS creado exitosamente.
💾 Vector store saved in ../vector_stores/teleqna/ft_vs_teleqna_150_20


## Train Phi-2

In [6]:
import sys
from utils.data_for_train_phi import get_dataset_for_train_phi
from vector_stores.faiss import VectorStoreFaiss
vector_store = VectorStoreFaiss.load_local("../vector_stores/teleqna/ft_vs_teleqna_150_20")
train_ds, test_ds = get_dataset_for_train_phi('teleqna', True, vector_store,3, 8)

Load model embedding : ../models/teleqna/embedding/bge-small-3gpp_10e_128bs/checkpoint-30
Using device: cuda
💾 Vector store loaded from../vector_stores/teleqna/ft_vs_teleqna_150_20
Creating dataset for teleqna
Loading dataset splits for teleqna
Train: 724
Val: 181
Test: 905
Datasets loaded and prepared.


🔍 Buscando: 100%|██████████████████████████| 23/23 [00:01<00:00, 15.96it/s]


Map:   0%|          | 0/181 [00:00<?, ? examples/s]

In [7]:
print(train_ds[0]['text'])

Instruct: Use the context provided below to answer the question. Select the correct option from A, B, C, D, E. Respond only with the the correct option. Do not include any explanations.
Context:

Document 0:1>	else if the failure is detected due to Mobility from NR failure as described in 5.4.3.5, set the fields in VarRLF-report as follows:

2>	set the connectionFailureType to hof;

2>	if last MobilityFromNRCommand concerned a failed inter-RAT handover from NR to E-UTRA and if the UE supports Radio Link Failure Report for Inter-RAT MRO EUTRA (NR to EUTRA):
Document 1:Release due to CN-detected mobility

The context release is requested by the AMF because the UE is already served by another CN node (same or different system), or another NG interface of the same CN node.

N26 interface not available

The action failed due to a temporary failure of the N26 interface.

Release due to pre-emption

Release is initiated due to pre-emption.

Multiple Location Reporting Reference ID Instances



In [8]:
!python  ../scripts/ft_phi.py \
  --new_model_name "phi-2-rag-k3_teleqna-5e_10bs" \
  --num_epochs 5 \
  --batch_size 10 \
  --dataset_name "teleqna" \
  --include_docs \
  --top_k 3 \
  --save_path "../models/teleqna/adapters/" \
  --vector_store_path "../vector_stores/teleqna/ft_vs_teleqna_150_20"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: dinho15971 (dinho15971-unicamp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.10
wandb: Run data is saved locally in /local1/ronaldinho/projects/test_sbbd/notebooks2/wandb/run-20250503_171222-0a2q7ld0
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run phi-2-rag-k3_teleqna-5e_10bs
wandb: ⭐️ View project at https://wandb.ai/dinho15971-unicamp/SBBD_phi-2-adapters
wandb: 🚀 View run at https://wandb.ai/dinho15971-unicamp/SBBD_phi-2-adapters/runs/0a2q7ld0
Usando dispositivo: cuda
Load model embedding : ../models/teleqna/embedding/bge-small-3gpp_10e_128bs/checkpoint-30
Using device: cuda
💾 Vector store loaded from../vector_stores/teleqna/ft_vs_teleqna_150_20
Using k = 3 passages
Creating dataset for teleqna
Loading dataset splits for teleqna
Train: 724
Val: 181
Test: 905
Datasets loaded and prepared.
🔍 Buscando: 100%|██████████████████████████████| 23/23 [00:01<00:00, 14.97it/s]


## Experiments 

In [9]:
!python ../scripts/inference_rag.py \
  --lora_adapter_path "../models/teleqna/adapters/best_phi-2-rag-k3_teleqna-5e_10bs" \
  --max_new_tokens 40 \
  --vector_store_path "../vector_stores/teleqna/ft_vs_teleqna_150_20" \
  --dataset_name "teleqna" \
  --output_csv_path "../results/teleqna/full_ft_k10_teleqna.csv" \
  --bs_emb 50 \
  --bs_gen 10 \
  --top_k 10 \
  --use_rag

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using device: cuda
Loading base model from LoRA adapter: ../models/teleqna/adapters/best_phi-2-rag-k3_teleqna-5e_10bs
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00,  3.60it/s]
Load model embedding : ../models/teleqna/embedding/bge-small-3gpp_10e_128bs/checkpoint-30
Using device: cuda
💾 Vector store loaded from../vector_stores/teleqna/ft_vs_teleqna_150_20
Device set to use cuda:0
Loading dataset splits for teleqna
Train: 724
Val: 181
Test: 905
Datasets loaded and prepared.
Dataset({
    features: ['question_id', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'explanation', 'category', 'tag'],
    num_rows: 905
})
Generating prompts: 100%|███████████████████| 905/905 [00:00<00:00, 6971.68it/s]
Example prompt
Instruct: Use the context provided below to answer the question. Select the correct option from A, B, C, D. Respond only with the the correct option. Do not include any explanations.
Context:

Document 1:The evaluation baseline includes at least NR R15 mandator

## Evaluate 

In [1]:
from utils.evaluate_inference import evaluate_answer
evaluate_answer('teleqna', '../results/teleqna/full_ft_k10_teleqna.csv')

Loading dataset splits for teleqna
Train: 724
Val: 181
Test: 905
Datasets loaded and prepared.
Results saved in: ../results/teleqna/full_ft_k10_teleqna.csv
